<a href="https://colab.research.google.com/github/AndreyBuyanov/HPC.Lab3/blob/main/HPC_Lab3_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['APACHE_SPARK_DISTR_NAME'] = 'spark-3.0.2-bin-hadoop2.7'
os.environ['APACHE_SPARK_URL'] = 'https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.0.2-bin-hadoop2.7'
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"andreew","key":"9237cca134821160d9d733ffd68a809c"}'}

In [ ]:
%%bash
apt-get install openjdk-11-jdk-headless -qq > /dev/null
[ ! -d $APACHE_SPARK_DISTR_NAME ] \
   && wget -q $APACHE_SPARK_URL \
   && tar xf $APACHE_SPARK_DISTR_NAME.tgz \
   && rm -f $APACHE_SPARK_DISTR_NAME.tgz
pip install findspark > /dev/null
mkdir /root/.kaggle
cp kaggle.json /root/.kaggle
chmod 600 /root/.kaggle/kaggle.json
[ ! -f FIFA.csv ] \
    && kaggle datasets download -d rgupta09/world-cup-2018-tweets \
    && unzip -q world-cup-2018-tweets \
    && rm -f world-cup-2018-tweets.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName("HPC.Lab3").getOrCreate()

In [ ]:
main_df = spark.read.csv('FIFA.csv', header=True, multiLine=True, escape="\"")
main_df.show(truncate=False)

+-------------------+----+-------------------+-------------------+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------+-----+----+-------------------------------------------------------+---------------------------------------------+----------------------------------------------------------+----------------------+----------------------+---------+-------+
|ID                 |lang|Date               |Source             |len|Orig_Tweet                                                                                                                                                            |Tweet                                                                                                                    |Likes|RTs |Hashtags                          

In [ ]:
# Функция сортирует словарь dict ПО ЗНАЧЕНИЮ
def sort_dict_by_value(dict, reverse=True):
    return {key: value for key, value in sorted(dict.items(), key=lambda x: x[1], reverse=reverse)}

In [ ]:
# Функция возвращает список первых n элементов из коллекции iterable
from itertools import islice
def take(n, iterable):
    return list(islice(iterable, n))

In [ ]:
def print_line(length=30):
    print('-' * length)

Создадим 5 временных промежутков

In [ ]:
import datetime as dt
month = 7
days = [1, 2, 3, 4, 10]
# Создадим список, элементами которого являются кортежи, состоящие из 2-х элементов (типа datetime): начало и конец временного промежутка
periods = [(dt.datetime(2018, month, day, 0, 0, 0), dt.datetime(2018, month, day, 23, 59, 59)) for day in days]
# Отобразим получившиеся временные промежутки
for period in periods:
    print(f'Period from {period[0]} to {period[1]}')

Period from 2018-07-01 00:00:00 to 2018-07-01 23:59:59
Period from 2018-07-02 00:00:00 to 2018-07-02 23:59:59
Period from 2018-07-03 00:00:00 to 2018-07-03 23:59:59
Period from 2018-07-04 00:00:00 to 2018-07-04 23:59:59
Period from 2018-07-10 00:00:00 to 2018-07-10 23:59:59


1. Определить десять наиболее упоминаемых хештегов для каждого выбранного временного промежутка

In [ ]:
#!!!!!!!!!!!! Этот блок кода выполняется достаточно долго
excluded_hashtag = 'WorldCup' # По условию задачи этот тег необходимо исключить
# Формат даты-времени в датасете
datetime_format='%Y-%m-%d %H:%M:%S'
# Количество хэштегов
hashtags_count = 10
# Будем хранить результат в списке
hashtags_by_period = []
# Проходим по каждому временному промежутку
for period in periods:
        # 1. map(lambda x: (dt.datetime.strptime(x['Date'], datetime_format), x['Hashtags']))
        # Преобразуем элементы. Из всего датасета нам нужны только 2 столбца: Date и Hashtags
        # map - применение заданной функции к каждой строке датасета и получение на выходе нового датасета
        # x - объект типа Row, x['Date'] - дата в виде строки, x['Hashtags'] - хэштеги в виде строки
        # Создадим рдд, состояший из одного столбца, элемент в ячейке - кортеж из двух элементов:
        # первый элемент - дата-время в виде объекта datetime (преобразуем на месте из строки. Это необходимо для сравнения, тк сравнивать строки так себе затея)
        # второй элемент - хэштеги в виде строки
        # 2. filter
        # Применяем фильтр: х - кортеж вида (datetime, строка_с_хэштегами)
        #                                      x[0]           x[1]
        # В итоге выбираем такие ячейки из рдд, дата которых больше начала временного промежутка и меньше конца временного промежутка
        # а также имеет непустое значение хэштегов (иначе код ниже, там где split вернёт ошибку, те игнорим строки без хэштегов)
        # 3. flatMap
        # Самое сложное как по мне это то, что в ячейке может быть несколько хэштегов, а нам они нужны по отдельности.
        # Поэтому выделим отдельные хэштеги, получаем список хэштегов - x[1].split(',')
        # Хэштеги из списка нужно поместить в отдельные строки, это можно сделать с помощью flatMap
        # Пример:
        # пусть у нас есть рдд с одним столбцом, содержащим строки, примерно такой
        # |---------
        # | Hashtags
        # |----------
        # | "tag1, tag2, tag3"
        # |------------
        # | "tag4, tag5"
        # |----------------
        # тогда x[1].split(',') вернет список строк
        # [ "tag1", "tag2", "tag3" ] и [ "tag4", "tag5" ]
        # те "как бы" получим такой рдд
        # |---------
        # | Hashtags
        # |----------
        # | [ "tag1", "tag2", "tag3" ]
        # |------------
        # | [ "tag4", "tag5" ]
        # |----------------
        # и теперь, если применить flatMap
        # то получим новый рдд, вот такой
        # |---------
        # | Hashtags
        # |----------
        # | "tag1"
        # |------------
        # | "tag2"
        # |------------
        # | "tag3"
        # |------------
        # | "tag4"
        # |------------
        # | "tag5"
        # |------------
        # те каждый элемент находится в отдельной ячейке
        # 4. filter(lambda x: x != excluded_hashtag)
        # Выбираем из рдд только ячейки, не содержащие excluded_hashtag
        # Важно: в итоговом рдд содержатся только хештеги, даты там нет, дата нужна была только для фильтрации
    period_rdd = main_df.rdd \
        .map(lambda x: (dt.datetime.strptime(x['Date'], datetime_format), x['Hashtags'])) \
        .filter(lambda x: x[0] > period[0] and x[0] < period[1] and x[1] != None) \
        .flatMap(lambda x: [hashtags for hashtags in x[1].split(',')]) \
        .filter(lambda x: x != excluded_hashtag)
    # !!! В целях отладки выведем несколько первых элементов полученного рдд
    print(f'DEBUG First 5 elements of rdd: {period_rdd.take(5)}')
    # Посчитаем, сколько раз встречается каждый элемент в рдд
    # Пример:
    # пусть у нас есть такой рдд
    # |----------
    # | "tag1"
    # |------------
    # | "tag2"
    # |------------
    # | "tag2"
    # |------------
    # | "tag2"
    # |------------
    # | "tag3"
    # |------------
    # Тогда countByValue вернёт словарь: ключ - элемент, значение - сколько раз этот элемент встречается
    # Получим такой словарь
    # {
    #    "tag1": 1,
    #    "tag2": 3,
    #    "tag3": 1,
    # }
    # Таким образом, мы посчитали, сколько раз каждый из тегов встречался в этом временном промежутке
    current_hashtags = period_rdd.countByValue()
    # Отсортируем полученный словарь по значению, те по количеству встречаемости тега
    # те в начале окажутся наиболее популярные хэштеги
    current_hashtags_sorted = sort_dict_by_value(current_hashtags)
    # а теперь просто возьмём первые 10 элементов словаря (ключей и значений), это и будут топ-10 хэштегов за данный промежуток.
    # Причём результат take - это список, в нашем случае список кортежей (ключей\значений исходного словаря - хэштегов\кол-во)
    current_hashtags_with_count = take(hashtags_count, current_hashtags_sorted.items())
    # !!! В целях отладки выведем топ-10 тегов
    print(f'DEBUG Top-10: {current_hashtags_with_count}')
    # Добавим к результату
    hashtags_by_period.append(current_hashtags_with_count)

DEBUG First 5 elements of rdd: ['DutchGP', 'GermanGP', '????????????', 'worldcup', 'RUS']
DEBUG Top-10: [('CRODEN', 11915), ('CRO', 9837), ('DEN', 7564), ('worldcup', 5379), ('FRAARG', 3998), ('FRA', 3992), ('POR', 3616), ('FIFAStadiumDJ', 3354), ('ARG', 3229), ('ESPRUS', 2910)]
DEBUG First 5 elements of rdd: ['POR', 'ENG', 'worldcup', 'PowerByEXO', 'FIFAStadiumDJ']
DEBUG Top-10: [('FIFAStadiumDJ', 1830), ('PowerByEXO', 1781), ('worldcup', 703), ('ESPRUS', 572), ('CRO', 541), ('CRODEN', 454), ('DEN', 452), ('ESP', 435), ('RUS', 414), ('LayLeague', 408)]
DEBUG First 5 elements of rdd: ['ENGCOL', 'ESP', 'ENG', 'ENG', 'ENG']
DEBUG Top-10: [('ENG', 8629), ('ThreeLions', 4945), ('COLENG', 1921), ('worldcup', 1871), ('ENGCOL', 1452), ('SWE', 685), ('BRA', 651), ('BEL', 611), ('FRA', 608), ('URU', 601)]
DEBUG First 5 elements of rdd: ['EXO', '??', 'FIFAStadiumDJ', 'Worldcup', 'PowerByEXO']
DEBUG Top-10: [('EXO', 2174), ('FIFAStadiumDJ', 2163), ('ENG', 1640), ('??', 1185), ('Worldcup', 789), (

In [ ]:
# hashtags_by_period - это список из 5 элементов (по количеству промежутков),
# элементы которого - это списки, элементы которых - кортежи (тег, количество)
# Выделим только хэштеги, без количества
hashtags = []
# Пройдём по результату
for hashtags_list in hashtags_by_period:
    # Преобразуем список из кортежей в список из строк (hashtag[0] - первый элемент кортежа - хэштег)
    current_hashtags = [hashtag[0] for hashtag in hashtags_list]
    # Добавим к результату
    hashtags.append(current_hashtags)

In [ ]:
hashtags # посмотрим, что там лежит

[['CRODEN',
  'CRO',
  'DEN',
  'worldcup',
  'FRAARG',
  'FRA',
  'POR',
  'FIFAStadiumDJ',
  'ARG',
  'ESPRUS'],
 ['FIFAStadiumDJ',
  'PowerByEXO',
  'worldcup',
  'ESPRUS',
  'CRO',
  'CRODEN',
  'DEN',
  'ESP',
  'RUS',
  'LayLeague'],
 ['ENG',
  'ThreeLions',
  'COLENG',
  'worldcup',
  'ENGCOL',
  'SWE',
  'BRA',
  'BEL',
  'FRA',
  'URU'],
 ['EXO',
  'FIFAStadiumDJ',
  'ENG',
  '??',
  'Worldcup',
  'ThreeLions',
  'dafBAMA2018EXO',
  'worldcup',
  'HereToCreate',
  'PowerByEXO'],
 ['FRABEL',
  'FRA',
  'BEL',
  'worldcup',
  'ENG',
  'France',
  'Belgium',
  'ThreeLions',
  'REDTOGETHER',
  'WorldCupFinal']]

Выведем результат

In [ ]:
print_line()
for i in range(len(periods)):
    print(f'Period from {periods[i][0]} to {periods[i][1]}, \ntop 10 hashtags: {hashtags[i]}')
    print_line()

------------------------------
Period from 2018-07-01 00:00:00 to 2018-07-01 23:59:59, 
top 10 hashtags: ['CRODEN', 'CRO', 'DEN', 'worldcup', 'FRAARG', 'FRA', 'POR', 'FIFAStadiumDJ', 'ARG', 'ESPRUS']
------------------------------
Period from 2018-07-02 00:00:00 to 2018-07-02 23:59:59, 
top 10 hashtags: ['FIFAStadiumDJ', 'PowerByEXO', 'worldcup', 'ESPRUS', 'CRO', 'CRODEN', 'DEN', 'ESP', 'RUS', 'LayLeague']
------------------------------
Period from 2018-07-03 00:00:00 to 2018-07-03 23:59:59, 
top 10 hashtags: ['ENG', 'ThreeLions', 'COLENG', 'worldcup', 'ENGCOL', 'SWE', 'BRA', 'BEL', 'FRA', 'URU']
------------------------------
Period from 2018-07-04 00:00:00 to 2018-07-04 23:59:59, 
top 10 hashtags: ['EXO', 'FIFAStadiumDJ', 'ENG', '??', 'Worldcup', 'ThreeLions', 'dafBAMA2018EXO', 'worldcup', 'HereToCreate', 'PowerByEXO']
------------------------------
Period from 2018-07-10 00:00:00 to 2018-07-10 23:59:59, 
top 10 hashtags: ['FRABEL', 'FRA', 'BEL', 'worldcup', 'ENG', 'France', 'Belgium

2. Определить, какие из выделенных хештегов уникальны для каждого промежутка. 

In [ ]:
# Вся работа ниже по сути уже никак не связана со спарком, всё делается на уровне встроенных средств питона для работы со множествами
unique_hashtags_by_period = []
# Пробежим по списку хэштегов
for current_period_hashtags in hashtags:
    # current_period_hashtags - список тегов для текущего временного промежутка
    # Создадим множество (те каждый элемент уникальный) из тегов из всех временных промежутков, за исключением тегов текущего временного промежутка
    all_hashtags_without_current = set.union(*[set(hashtag) for hashtag in hashtags if hashtag != current_period_hashtags])
    # Превратим список тегов для текущего временного промежутка в множество
    current_period_hashtags_set = set(current_period_hashtags)
    # Вычислим разность множеств, те из всего множества тегов выберем те, которые встречаются только в данном временном промежутке
    # Загугли диаграммы Эйлера-Венна для разности множеств и поймёшь идею
    # Если непонятно, добавь print, чтобы увидеть множества и что с ними происходит
    unique_hashtags = current_period_hashtags_set - all_hashtags_without_current
    # Добавим к результату
    unique_hashtags_by_period.append(unique_hashtags)

Выведем результат

In [ ]:
print_line()
for i in range(len(periods)):
    print(f'Period from {periods[i][0]} to {periods[i][1]}, unique hashtags: {unique_hashtags_by_period[i]}')
    print_line()

------------------------------
Period from 2018-07-01 00:00:00 to 2018-07-01 23:59:59, unique hashtags: {'POR', 'FRAARG', 'ARG'}
------------------------------
Period from 2018-07-02 00:00:00 to 2018-07-02 23:59:59, unique hashtags: {'ESP', 'LayLeague', 'RUS'}
------------------------------
Period from 2018-07-03 00:00:00 to 2018-07-03 23:59:59, unique hashtags: {'COLENG', 'BRA', 'SWE', 'ENGCOL', 'URU'}
------------------------------
Period from 2018-07-04 00:00:00 to 2018-07-04 23:59:59, unique hashtags: {'HereToCreate', '??', 'dafBAMA2018EXO', 'Worldcup', 'EXO'}
------------------------------
Period from 2018-07-10 00:00:00 to 2018-07-10 23:59:59, unique hashtags: {'Belgium', 'France', 'FRABEL', 'WorldCupFinal', 'REDTOGETHER'}
------------------------------


3. Определить, для каких двух промежутков наиболее близкое друг к другу соотношение частоты встречаемости хештегов.

In [ ]:
# Найдём топ-10 тегов. "частоты встречаемости хештегов" - каких? всех? непонятно, поэтому и берём топ-10
# тут ничего нового, см код из 1-го задания
all_hashtags_with_count_rdd = main_df.rdd \
    .map(lambda x: x['Hashtags']) \
    .filter(lambda x: x != None) \
    .flatMap(lambda x: [hashtags for hashtags in x.split(',')]) \
    .filter(lambda x: x != excluded_hashtag)
# посчитаем, сколько раз каждый из тегов встречался в датасете
all_hashtags_with_count = all_hashtags_with_count_rdd \
    .countByValue()
# Отсортируем полученный словарь по значению, те по количеству встречаемости тега
# те в начале окажутся наиболее популярные хэштеги
all_hashtags_with_count_sorted = sort_dict_by_value(all_hashtags_with_count)
# а теперь просто возьмём первые 10 элементов словаря (ключей и значений), это и будут топ-10 хэштегов из всего датасета
top10_hashtags_with_count = take(10, all_hashtags_with_count_sorted.items())
# теперь количество встречаемости тега нам не нужно, избавимся от количества, создав новый список, в котором будут только теги
top10_hashtags = [hashtag for hashtag, count in top10_hashtags_with_count]
print(f'Top 10 hashtags: {", ".join(top10_hashtags)}')

Top 10 hashtags: FRA, worldcup, CRO, WorldCupFinal, ENG, FRAARG, FRABEL, FRACRO, BEL, ARG


In [ ]:
# словарь: ключ - период (кортеж из двух элементов, начало и конец периода)
# значение - словарь: ключ - тег из топ10, значение - количество упоминаний этого тега за данный период
all_hashtags_with_count_by_period = {}
# бежим по периодам
for period in periods:
    # создаём новый рдд, очень похожий на тот, что из первого задания
    # однако тут добавляем ещё один фильтр:
    # filter(lambda x: top10_hashtags.count(x) == 1)
    # те из всего множества тегов мы берём только те, что из топ-10
    # top10_hashtags.count(x) == 1 - top10_hashtags - это список (см. выше), х - текущий тег
    # и если он есть в списке (== 1), то окей, берём этот тег, иначе игноригуем
    period_rdd = main_df.rdd \
        .map(lambda x: (dt.datetime.strptime(x['Date'], datetime_format), x['Hashtags'])) \
        .filter(lambda x: x[0] > period[0] and x[0] < period[1] and x[1] != None) \
        .flatMap(lambda x: [hashtags for hashtags in x[1].split(',')]) \
        .filter(lambda x: x != excluded_hashtag) \
        .filter(lambda x: top10_hashtags.count(x) == 1)
    # посчитаем, сколько раз каждый из тегов встречался в этом рдд (те во текущем временном промежутке)
    all_hashtags_with_count = period_rdd \
        .countByValue()
    # складируем результат
    all_hashtags_with_count_by_period[period] = all_hashtags_with_count
    print(all_hashtags_with_count) # это для отладки

defaultdict(<class 'int'>, {'worldcup': 5379, 'CRO': 9837, 'BEL': 332, 'FRAARG': 3998, 'FRA': 3992, 'ARG': 3229, 'ENG': 1286, 'WorldCupFinal': 3})
defaultdict(<class 'int'>, {'ENG': 92, 'worldcup': 703, 'ARG': 141, 'CRO': 541, 'FRA': 312, 'FRAARG': 212, 'BEL': 148})
defaultdict(<class 'int'>, {'ENG': 8629, 'worldcup': 1871, 'BEL': 611, 'FRA': 608, 'CRO': 278, 'ARG': 43, 'FRAARG': 4, 'WorldCupFinal': 1})
defaultdict(<class 'int'>, {'ENG': 1640, 'worldcup': 632, 'CRO': 54, 'FRA': 127, 'BEL': 129, 'ARG': 27, 'FRAARG': 4})
defaultdict(<class 'int'>, {'FRA': 11118, 'ENG': 3876, 'FRABEL': 18750, 'BEL': 7132, 'worldcup': 4678, 'CRO': 1414, 'ARG': 28, 'WorldCupFinal': 1627, 'FRAARG': 7, 'FRACRO': 3})


In [ ]:
# Выводим результат
print_line()
# бежим по топу хештегов
for hashtag in top10_hashtags:
    print(f'Hashtag "{hashtag}"')
    # временная переменная
    hashtags_count_in_period = {}
    # бежим по словарю
    for period, hashtags in all_hashtags_with_count_by_period.items():
        # сохраняем в словаре по ключу "текущий период" количество упоминаний данного тега в текущем периоде
        hashtags_count_in_period[period] = hashtags[hashtag] if hashtag in hashtags else 0 # хэштэга может не оказаться, поэтому проверяем есть ли количество упоминаний
    # отсортируем результат по занчению (количеству упоминаний)
    hashtags_count_in_period_sorted = sort_dict_by_value(hashtags_count_in_period)
    # возьмём первые два элемента словаря - это и будут два самых упоминаемых хэштега
    first2_elements = take(2, hashtags_count_in_period_sorted.items())
    # извлекём данные этих двух элементов
    (period1_start, period1_end), frequency1 = first2_elements[0]
    (period2_start, period2_end), frequency2 = first2_elements[1]
    # красивенько (или не очень) выведем результат
    print(f'Period 1: {period1_start.strftime(datetime_format)} - {period1_end.strftime(datetime_format)}, frequency {frequency1}')
    print(f'Period 2: {period2_start.strftime(datetime_format)} - {period2_end.strftime(datetime_format)}, frequency {frequency2}')
    print_line()
# таким образом мы нашли по два периода для каждого из тегов из топ-10, в которых данный тег упоминался чаще всего

------------------------------
Hashtag "FRA"
Period 1: 2018-07-10 00:00:00 - 2018-07-10 23:59:59, frequency 11118
Period 2: 2018-07-01 00:00:00 - 2018-07-01 23:59:59, frequency 3992
------------------------------
Hashtag "worldcup"
Period 1: 2018-07-01 00:00:00 - 2018-07-01 23:59:59, frequency 5379
Period 2: 2018-07-10 00:00:00 - 2018-07-10 23:59:59, frequency 4678
------------------------------
Hashtag "CRO"
Period 1: 2018-07-01 00:00:00 - 2018-07-01 23:59:59, frequency 9837
Period 2: 2018-07-10 00:00:00 - 2018-07-10 23:59:59, frequency 1414
------------------------------
Hashtag "WorldCupFinal"
Period 1: 2018-07-10 00:00:00 - 2018-07-10 23:59:59, frequency 1627
Period 2: 2018-07-01 00:00:00 - 2018-07-01 23:59:59, frequency 3
------------------------------
Hashtag "ENG"
Period 1: 2018-07-03 00:00:00 - 2018-07-03 23:59:59, frequency 8629
Period 2: 2018-07-10 00:00:00 - 2018-07-10 23:59:59, frequency 3876
------------------------------
Hashtag "FRAARG"
Period 1: 2018-07-01 00:00:00 - 20